## Experience Analytics

In [1]:
# Importing useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
# Libraries used to connect to database
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import psycopg2

In [ ]:
# Adjust the path as needed based on the notebook's location relative to the src directory
import sys
sys.path.append("../src")  

In [ ]:
# Helper functions

# Data loading function from the src.loader module
from loader import connect_to_database